In [1]:
import pandas as pd
import torch
import gc
from numba import cuda
from tqdm import tqdm
import json
from datasets import Dataset
import sys
import re
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
import random

from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import TextClassificationPipeline
from transformers import AutoTokenizer,AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import pipeline as transformers_pipeline
from transformers import T5Tokenizer, T5ForConditionalGeneration


2024-12-20 09:00:16.968237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734685216.984107 1872806 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734685216.988814 1872806 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 09:00:17.009574: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#IMPORT AND PREPARE DATASET

path = ""  # Replace with your dataset path
ds = pd.read_csv(path)

dataset = Dataset.from_pandas(ds)

print(dataset)

Dataset({
    features: ['Id', 'Text', 'Label'],
    num_rows: 3901
})


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-xl',device_map='auto')
tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-xl')
model_name = 'google/flan-t5-xl'

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [4]:
#IMPORT DEFINTIONS - STEP 1
def_step_1 = {  "NO":"",
                "MHS":"Hate Speech defined as, bias-motivated, hostile and malicious language targeted at a person/group because of their actual or perceived innate characteristics, especially when the group is unnecessarily labeled",
                "OL":"Hate Speech is considered any kind of content that conveys malevolent intentions toward a group or an individual.",
                "HSB":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that convey malevolent intentions toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDPC":"Hate speech is considered any kind of content that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDT":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDT":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that convey malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDPC":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions, that convey malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDT_EDPC":"Hate speech is considered any kind of content that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members.",
                "HSB_EDFoC_EDPC_EDT":"Hate speech is considered any kind of content or communication expressed using language (written or spoken) or actions,  that conveys malevolent intentions such as statements of inferiority, aversion, cursing, calls for exclusion, threaten, harass or violence, toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members."
                }

def_step_2 = {"HSB_EDT_LAA":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics.",
            "HSB_EDT_LAA_PI":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety.",
            "HSB_EDT_LAA_Exc":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions",
            "HSB_EDT_LAA_IHS":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation.",
            "HSB_EDT_LAA_PI_Exc":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions.",
            "HSB_EDT_LAA_Exc_IHS":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions.",
            "HSB_EDT_LAA_PI_IHS":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation.",
            "HSB_EDT_LAA_PI_IHS_Exc":"Hate speech is considered any kind of content that conveys malevolent intentions toward a group or an individual which is, or thought to be, a member of that group, and motivated by inherent characteristics that are attributed to that group and shared among its members such as race, color, ethnicity, gender, sexual orientation, nationality, religion, disability, social status, health conditions, or other characteristics. The outcome of Hate Speech could be the promotion of division among people, undermining of social cohesion in communities, inciting others to commit violence or discrimination, and could have consequences for individuals’ health and safety. Hate Speech can also be implicit, portrayed as an indirect or coded language that uses Irony, Stereotypes, or Misinformation. However, even if it is offensive, it is not considered Hate Speech any content that attacks a person’s personality traits, ideas, or opinions."
            }

In [5]:
model_inputs = []
labels = []
instructions = "Classify if the following text is considered hate speech or not hate speech:\n"\
              "[TEXT]\n"\
              "Please answer ONLY with 1 if it is hate speech or 0 if it is not hate speech."
        

for datapoint in tqdm(dataset):
    model_input = instructions
    model_input = model_input.replace("[TEXT]", datapoint['Text'])
    if datapoint['Label'] == 1:
        labels.append(1)
    else:
        labels.append(0)
    model_inputs.append(model_input)
print(model_inputs[1])

100%|██████████| 3901/3901 [00:00<00:00, 53725.98it/s]

Classify if the following text is considered hate speech or not hate speech:
Let's send back the white trash to their origins then we can send Ilhan
Please answer ONLY with 1 if it is hate speech or 0 if it is not hate speech.


In [6]:
def metriche(predictions,labels):

    predictions = [np.nan if x == 'NaN' else x for x in predictions]

    #convert predictions to numpy array to handle NaN values
    predictions = np.array(predictions, dtype=float)  

    #replace NaN values with a default value (e.g., 0) if needed
    predictions[np.isnan(predictions)] = 0  

    precision = precision_score(labels, predictions, average='binary', zero_division=0)

    recall = recall_score(labels, predictions, average='binary', zero_division=0)

    f1 = f1_score(labels, predictions, average='macro', zero_division=0)
    
    return precision, recall, f1
    

In [7]:
from tqdm import tqdm

def runner(model_inputs, labels, num_runs, id_def, definition, sample):
    results = []

    for j in range(num_runs):  
        if sample == 'all' or sample >= 3901:
            sample = 3901
        else:
            model_inputs = model_inputs[:sample]
            labels = labels[:sample]
    
        completions = []
        responses = []

        system_prompt = ''

        model_prompt =  "[SYSTEM_PROMPT] \n [USER_MESSAGE] \n"

        for i, inputtone in enumerate(tqdm(model_inputs)): 
            if id_def == 'NO':
                tot_in = inputtone
            else:
                tot_in = 'Given the following definition of Hate Speech: \n' + str(definition) +'\n' + inputtone 

            model_in = model_prompt.replace("[SYSTEM_PROMPT]", system_prompt)
            model_in = model_in.replace("[USER_MESSAGE]", tot_in)

            # Tokenize the input using the tokenizer
            input_ids = tokenizer(model_in, return_tensors="pt").input_ids.to("cuda")


            # Generate outputs from the model
            outputs = model.generate(input_ids,temperature=0.95, do_sample = True)

            # Decode the output IDs to text
            decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = decoded_output
            responses.append(response)


            predictions = []
            for response in responses:
                if response == '1':
                    predictions.append(1)
                elif response == '0':
                    predictions.append(0)
                else:
                    predictions.append('NaN')

        num = j+1
        print('Run number:', num, 'With definition: ', id_def)     
        numeratore = sum(1 for p,l in zip(predictions,labels) if p ==l)
        denominatore = len(predictions)
        acc = numeratore/denominatore
        print('Accuracy: ', acc)
        nans = sum(1 for p in predictions if p =='NaN')
        print('Numbers of non-answer: ',nans)
        if denominatore == nans:
            acc_no_nans = 0
        else:
            acc_no_nans = numeratore/(denominatore-nans)
        print('Accuracy without no-answers: ', acc_no_nans)

        precision, recall, f1 = metriche(predictions, labels)

        results.append({'Run': num,
                            'Model': model_name, 
                            'ID_def': id_def,
                            'Accuracy': acc, 
                            'NaNs': nans, 
                            'Precision': precision, 
                            'Recall': recall, 
                            'F1': f1 ,
                            'Input': dataset['Text'],
                            'Responses': responses,
                            'Predictions': predictions, 
                            'Labels': labels, 
                            'Definition': definition})

        
        print('Accuracy: ', acc)
        print('Precision: ', precision)
        print('Recall: ', recall)
        print('F1 Score: ', f1)
        
    return results



In [ ]:
#ALL THE DEFINITIONS
sample = 'all' #select a number of samples or "all" for the whole dataset
num_runs = 3  #number of iterations 
step = '1' # 1 for step 1, 2 for step 2

results = []

if step == '1':
    def_dict = def_step1
    step_name = 'step1'
elif step == '2':
    def_dict = def_step2
    step_name = 'step2'

    
for k, v in def_dict.items():
    def_id = k
    definition = v
    result = runner(model_inputs, labels, num_runs=num_runs, id_def=def_id, definition=definition, sample=sample)
    results.extend(result) 

In [9]:
df = pd.DataFrame(results)

csv_file_path = '../Outputs/Flant5_mhs_'+ step_name +'.csv'

df.to_csv(csv_file_path, index=False)  

In [10]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = 'Flant5_mhs_step1.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Group the DataFrame by 'id_def' and calculate the mean of 'Accuracy' within each group
avg_accuracy_per_id = df.groupby('ID_def')['F1'].mean().reset_index()

avg_accuracy_per_id_sorted = avg_accuracy_per_id.sort_values(by='F1', ascending=False)

# Display the average accuracy for each id_def
print(avg_accuracy_per_id_sorted)

                ID_def        F1
6              HSB_EDT  0.755434
4        HSB_EDFoC_EDT  0.753827
1            HSB_EDFoC  0.752089
3   HSB_EDFoC_EDPC_EDT  0.751469
2       HSB_EDFoC_EDPC  0.748467
8                  MHS  0.747939
5             HSB_EDPC  0.747651
10                  OL  0.746298
7         HSB_EDT_EDPC  0.745792
0                  HSB  0.742970
9                   NO  0.742132
